# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [8]:

from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.metrics import *
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import pymorphy2
import pandas as pd
import nltk
import random
from functools import reduce
nltk.download('wordnet')
nltk.download('stopwords')
stemmer = SnowballStemmer("russian")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\reser\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\reser\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [4]:
!!pip install Levenshtein
import Levenshtein

# Чтение слов из файла и запись их в список words
with open('litw-win.txt', 'r') as file:
    words = file.read().split()

text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

# Разделение текста на слова
text_words = text.split()

# Замена опечаток в тексте
corrected_text = []
for word in text_words:
    if word not in words:
        closest_word = min(words, key=lambda x: Levenshtein.distance(word, x))
        corrected_text.append(closest_word)
    else:
        corrected_text.append(word)

# Объединение слов в исправленный текст
corrected_text = ' '.join(corrected_text)

print(corrected_text)


с величайшим усилием выбравшись из потока убегающих людей кутузов со свитой уменьшившейся вдвое поехал на звуки выстрелов русских орудий


2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [13]:
import nltk
from nltk.stem import SnowballStemmer, WordNetLemmatizer

text =  '''
        Считайте слова из файла litw-win.txt и запишите их в список words. 
        В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна)
        к ним слова из списка words. Считайте, что в слове есть опечатка, если данное слово не содержится в списке words
        '''
words = nltk.word_tokenize(text)
stemmer = SnowballStemmer("russian")
stemmed_words = [stemmer.stem(word) for word in words]
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
stemmed_words
lemmatized_words



['Считайте',
 'слова',
 'из',
 'файла',
 'litw-win.txt',
 'и',
 'запишите',
 'их',
 'в',
 'список',
 'word',
 '.',
 'В',
 'заданном',
 'предложении',
 'исправьте',
 'все',
 'опечатки',
 ',',
 'заменив',
 'слова',
 'с',
 'опечатками',
 'на',
 'ближайшие',
 '(',
 'в',
 'смысле',
 'расстояния',
 'Левенштейна',
 ')',
 'к',
 'ним',
 'слова',
 'из',
 'списка',
 'word',
 '.',
 'Считайте',
 ',',
 'что',
 'в',
 'слове',
 'есть',
 'опечатка',
 ',',
 'если',
 'данное',
 'слово',
 'не',
 'содержится',
 'в',
 'списке',
 'word']

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

In [17]:

from sklearn.feature_extraction.text import CountVectorizer

text =  '''
        Считайте слова из файла litw-win.txt и запишите их в список words. 
        В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна)
        к ним слова из списка words. Считайте, что в слове есть опечатка, если данное слово не содержится в списке words
        '''

vectorizer = CountVectorizer()
X = vectorizer.fit_transform([text])
print(X.toarray())

[[1 1 1 3 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 2 1 1]]


## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [5]:
import pandas as pd
import random
from nltk.tokenize import word_tokenize
from functools import reduce

n = random.randint(301, 500)
preprocessed_descriptions = pd.read_csv("preprocessed_descriptions.csv")[:n]
preprocessed_descriptions.head()
words = reduce(lambda x, y: x + y, [word_tokenize(item) for item in preprocessed_descriptions["preprocessed_descriptions"].to_list() if isinstance(item, str)])
print(words[:10])


['an', 'original', 'recipe', 'created', 'by', 'chef', 'scott', 'meskan', 'george', 's']


1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [8]:
words = list(set(words))
pairs = [' '.join(random.choices(words, k=2)) for _ in range(5)]
print(pairs)
h_s = [edit_distance(*v.split()) for v in pairs]
h_s

['inexpensive races', 'browsing cassava', 'learn aduki', 'wrapped blogspot', 'mixes dare']


[9, 7, 5, 7, 4]

1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [9]:
def k_nearest(word:str, k:int=1):
    w_new = sorted(words, key=lambda w: edit_distance(w, word))
    return w_new[:k]

k_nearest('check', k=3)

['check', 'chuck', 'chef']

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [10]:
lemmatizer = WordNetLemmatizer()

stemmed_words = [stemmer.stem(word) for word in words]
normalized_word = [lemmatizer.lemmatize(word) for word in words]
df = pd.DataFrame(list(zip(words, stemmed_words, normalized_word))[10:20], columns=['word', 'stemmed_word', 'normalized_word'])
df = df.set_index('stemmed_word')
df.head()

,word,normalized_word
stemmed_word,,
working,working,working
reportedly,reportedly,reportedly
runny,runny,runny
tenderloin,tenderloin,tenderloin
coriander,coriander,coriander


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [11]:

st_w = stopwords.words()
texts = reduce(lambda x, y: x + ' ' + y, [item for item in preprocessed_descriptions["preprocessed_descriptions"].to_list() if isinstance(item, str)])
tokens = word_tokenize(texts)
l = len(tokens)
l_new = 0
words_dict = {}
words_dict_stop = {}
for w in tokens:
    if w not in st_w:
        words_dict_stop[w] = words_dict_stop.get(w, 0) + 1
        l_new += 1
    words_dict[w] = words_dict.get(w, 0) + 1


print(f'Доля стоп слов - {(l - l_new) / l}')
print(f'Топ 10 слов со стоп словами: {"; ".join(sorted(words_dict.keys(), key=lambda x: words_dict[x], reverse=True)[:10])}')
print(f'Топ 10 слов без стоп слов: {"; ".join(sorted(words_dict_stop.keys(), key=lambda x: words_dict_stop[x], reverse=True)[:10])}')

Доля стоп слов - 0.5835470197289235
Топ 10 слов со стоп словами: the; a; i; and; this; it; to; is; of; for
Топ 10 слов без стоп слов: recipe; make; easy; great; time; made; dish; delicious; bread; chicken


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [12]:
data = preprocessed_descriptions.sample(5)
print(data)

vectorizer = TfidfVectorizer()
vectorizer.fit(data['preprocessed_descriptions'])
sent_vec = vectorizer.transform(data['preprocessed_descriptions'])
sent_vec = sent_vec.toarray()
for i, recipe in enumerate(data['preprocessed_descriptions']):
    print("Рецепт:\n", recipe)
    print("Вектор:\n", sent_vec[i])
    print()

                                         name  \
383                            afterburner  3   
413                   alaska molasses cookies   
63     reasonable  lemon chicken and potatoes   
156                       3 cheese baked ziti   
297  absolute best chicken tortilla soup ever   

                             preprocessed_descriptions  
383  a drink not too sweet   in this drink  we can ...  
413  from cooking light  september 2002    allow an...  
63   this is a recipe my mother and i made up it is...  
156                          i got this from a ragu ad  
297  i got this recipe from a website for which i v...  
Рецепт:
 a drink not too sweet   in this drink  we can taste the vodka and triple sec   

http   www drinksmixer com drink754 html
Вектор:
 [0.         0.         0.         0.         0.         0.
 0.         0.17781164 0.         0.         0.         0.
 0.         0.22039294 0.         0.         0.22039294 0.
 0.         0.         0.         0.         0

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [13]:
import itertools
from scipy.spatial import distance

max_pair = None
max_result = -1

coeff_dict = {}
vectorizer3 = TfidfVectorizer(analyzer="word", stop_words="english")
transform3 = vectorizer3.fit_transform(data["preprocessed_descriptions"].to_list())

all_data = list(zip(data["preprocessed_descriptions"].to_list(), transform3.toarray()))

for pair in itertools.product(all_data, repeat=2):

    text1, matrix1 = pair[0]
    text2, matrix2 = pair[1]
    result = distance.cosine(matrix1, matrix2)
    inverse_result = 1-result

    if text1 not in coeff_dict:
        coeff_dict[text1] = []
    coeff_dict[text1].append(inverse_result)


    if inverse_result > max_result and text1 != text2:
        max_result = inverse_result
        max_pair = (text1, text2)

    print(f"{text1}\n{text2}\n{inverse_result}\n")

df_final2 = pd.DataFrame.from_dict(coeff_dict)
df_final2.columns = data["preprocessed_descriptions"].to_list()
df_final2.index = data["preprocessed_descriptions"].to_list()
df_final2

a drink not too sweet   in this drink  we can taste the vodka and triple sec   

http   www drinksmixer com drink754 html
a drink not too sweet   in this drink  we can taste the vodka and triple sec   

http   www drinksmixer com drink754 html
1

a drink not too sweet   in this drink  we can taste the vodka and triple sec   

http   www drinksmixer com drink754 html
from cooking light  september 2002    allow an extra 30 minutes for the dough to chill 
0.0

a drink not too sweet   in this drink  we can taste the vodka and triple sec   

http   www drinksmixer com drink754 html
this is a recipe my mother and i made up it is both easy to make and delicious   it is the most tender chicken i ve ever had   if you like things less sour  use a little less lemon juice 
0.0

a drink not too sweet   in this drink  we can taste the vodka and triple sec   

http   www drinksmixer com drink754 html
i got this from a ragu ad
0.0

a drink not too sweet   in this drink  we can taste the vodka and trip

,a drink not too sweet in this drink we can taste the vodka and triple sec \r\n\r\nhttp www drinksmixer com drink754 html,from cooking light september 2002 allow an extra 30 minutes for the dough to chill,this is a recipe my mother and i made up it is both easy to make and delicious it is the most tender chicken i ve ever had if you like things less sour use a little less lemon juice,i got this from a ragu ad,i got this recipe from a website for which i ve long since lost the address i wish i could credit the author because this is truly an amazing soup i d recommend not skipping the avocados they are what puts it over the top in my opinion
a drink not too sweet in this drink we can taste the vodka and triple sec \r\n\r\nhttp www drinksmixer com drink754 html,1.0,0.0,0.000000,0.000000,0.000000
from cooking light september 2002 allow an extra 30 minutes for the dough to chill,0.0,1.0,0.000000,0.000000,0.000000
this is a recipe my mother and i made up it is both easy to make and delicious it is the most tender chicken i ve ever had if you like things less sour use a little less lemon juice,0.0,0.0,1.000000,0.000000,0.083606
i got this from a ragu ad,0.0,0.0,0.000000,1.000000,0.097097
i got this recipe from a website for which i ve long since lost the address i wish i could credit the author because this is truly an amazing soup i d recommend not skipping the avocados they are what puts it over the top in my opinion,0.0,0.0,0.083606,0.097097,1.000000


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

In [14]:
print(f"Из датасета выше больше всего совпадений в предложениях:\n\n{max_pair[0]}\n\n{max_pair[1]}\n\n{max_result}")

Из датасета выше больше всего совпадений в предложениях:

i got this from a ragu ad

i got this recipe from a website for which i ve long since lost the address   i wish i could credit the author  because this is truly an amazing soup   i d recommend not skipping the avocados  they are what puts it over the top  in my opinion 

0.09709725545237846
